# Evaulation

This notebook is setup to help evaluate the runs

In [1]:
%matplotlib inline
from dotmap import DotMap
from envs import *
import os
import dill as pickle
import run_learning
import numpy as np
import run_eval
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [20, 10]
from stable_baselines3 import SAC

# Find Desired Runs

In [2]:
def look_through_folders():
    for folder in os.listdir("./Runs"):
        folder_param_path = os.path.join(os.path.join("./Runs", folder), "params.pkl")
        if os.path.exists(folder_param_path):
            with open(os.path.join(os.path.join("./Runs", folder), "params.pkl"), 'rb') as f:
                params = pickle.load(f)
            print("Eps: ", str(params.eps), ", Gamma: ", str(params.gamma), ", LearningRate: ", str(params.learning_rate))
            print("Eval_Freq: ", str(params.init_state))
#look_through_folders()

In [13]:
def check(params):
    match = bool(1
    #and params.timesteps == 275000
    #and params.gamma == 0.98
    and params.id == 4
    #and params.use_sde == True
    #and params.eps == 1
    #and params.learning_rate == 0.0003
    #and params.envs.newpendulum.run == True
    )
    return match

def filt(check):
    matches = []
    for folder in os.listdir("./Runs"):
        folder_param_path = os.path.join(os.path.join("./Runs", folder), "params.pkl")
        if os.path.exists(folder_param_path):
            with open(os.path.join(os.path.join("./Runs", folder), "params.pkl"), 'rb') as f:
                params = pickle.load(f)
            if check(params):
                matches.append(folder)
    return matches

a = ['02_17_2022_083556_Mohsin', '02_17_2022_074751_Mohsin', '02_17_2022_043715_Mohsin']
filt(check)

['02_23_2022_174221_Mohsin',
 '02_23_2022_095401_Mohsin',
 '02_23_2022_020636_Mohsin',
 '02_22_2022_214047_Mohsin',
 '02_23_2022_191043_Mohsin',
 '02_23_2022_082408_Mohsin',
 '02_22_2022_200325_Mohsin',
 '02_23_2022_112214_Mohsin',
 '02_22_2022_182721_Mohsin',
 '02_23_2022_161317_Mohsin',
 '02_22_2022_230934_Mohsin',
 '02_23_2022_003810_Mohsin',
 '02_22_2022_165045_Mohsin',
 '02_23_2022_143640_Mohsin',
 '02_23_2022_125859_Mohsin',
 '02_23_2022_065656_Mohsin',
 '02_23_2022_034312_Mohsin',
 '02_23_2022_052020_Mohsin']

# Evaluate Desired Runs

In [8]:
label = "eps"
def avg(folders):
    tb_times_tot = None
    theta_tot = []
    actions_tot = [] 
    tb_rew_tot = []
    plt.figure()
    for folder in folders:
        results = run_eval.evaluate(folder, init=[3, 0])
        tb = results["newpendulum"]["mean_reward"]
        tb_times = tb[0]
        tb_rew = [a[0] for a in tb[1]]
        theta = results["newpendulum"]["thetas"]
        actions = results["newpendulum"]["actions"]
        
        tb_times_tot = tb_times
        theta_tot.append(theta)
        actions_tot.append(actions)
        tb_rew_tot.append(tb_rew)
        
    theta_avg = np.average(theta_tot, axis=0)
    actions_avg = np.average(actions_tot, axis=0)
    tb_rew_avg = np.average(tb_rew_tot, axis=0)

    with open(os.path.join(os.path.join("./Runs", folder), "params.pkl"), 'rb') as f:
        params = pickle.load(f)
        
    plt.subplot(122)
    #plt.plot(tb_times, tb_rew, label=str(params[label]))
    plt.plot(tb_times, tb_rew_avg, label=str(params[label]))
    plt.subplot(221)
    plt.plot(theta_avg, label=str(params[label]))
    #plt.ylim([-4, 4])
    plt.subplot(223)
    plt.plot(actions_avg, label=str(params[label]))   

    plt.subplot(122)
    plt.ylabel("reward")
    plt.legend()
    plt.subplot(221)
    plt.ylabel("theta")
    plt.legend()
    plt.subplot(223)
    plt.ylabel("actions")
    plt.legend()
    plt.show()
    
    
avg(filt(check))

KeyError: 'newpendulum'

<Figure size 1440x720 with 0 Axes>

In [14]:
label = "eps"

def plot_eval(folders):
    plt.figure()
    for folder in folders:
        results = run_eval.evaluate(folder, init=[3, 0])
        tb = results["newpendulum"]["mean_reward"]
        tb_times = tb[0]
        tb_rew = [a[0] for a in tb[1]]
        
        n=101
        tb_smoothed = np.cumsum(tb_rew, dtype=float)
        tb_smoothed[n:] = tb_smoothed[n:] - tb_smoothed[:-n]
        tb_smoothed =  tb_smoothed[n - 1:] / n

        theta = results["newpendulum"]["thetas"]
        actions = results["newpendulum"]["actions"]
        with open(os.path.join(os.path.join("./Runs", folder), "params.pkl"), 'rb') as f:
            params = pickle.load(f)
        plt.subplot(122)
        #plt.plot(tb_times, tb_rew, label=str(params[label]))
        plt.plot(tb_times[50:-50], tb_smoothed, label=str(params[label]))
        plt.subplot(221)
        plt.plot(theta, label=str(params[label]))
        #plt.ylim([-4, 4])
        plt.subplot(223)
        plt.plot(actions, label=str(params[label]))   
        
    plt.subplot(122)
    plt.ylabel("reward")
    plt.legend()
    plt.subplot(221)
    plt.ylabel("theta")
    plt.legend()
    plt.subplot(223)
    plt.ylabel("actions")
    plt.legend()
    plt.show()
        
plot_eval(filt(check))

AttributeError: 'Box' object has no attribute 'shape'

<Figure size 1440x720 with 0 Axes>

# Rendering Station

DON"T USE YET, BEHAVIOR NOT WORKING

In [10]:
folder = '02_14_2022_123836_Mohsin'
env_name = 'newpendulum'
model_name = 'best_model'
with open(os.path.join(os.path.join("./Runs", folder), "params.pkl"), 'rb') as f:
            params = pickle.load(f)
env = params.envs[env_name].eval_env(params)
env.reset()
models_path = os.path.join(os.path.join("./Runs", folder), env_name + "/models")
model = SAC.load(os.path.join(models_path, model_name))

obs = env.reset()
done = False
i = 0
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    #env.render()

/opt/anaconda3/envs/rl/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


UnboundLocalError: local variable 'costs' referenced before assignment